
# SUPPORT Notebook

This notebook runs through investigations on the open access SUPPORT dataset.

For users who do not have lots of computational resources or do not have access to MIMIC-III then this notebook should be used.

In [1]:
#%% -------- Import Libraries -------- #

# Standard imports
import numpy as np
import pandas as pd
import torch

# VAE is in other folder as well as opacus adapted library
import sys
sys.path.append('../')

# Opacus support for differential privacy
from opacus.utils.uniform_sampler import UniformWithReplacementSampler

# For the SUPPORT dataset
from pycox.datasets import support

# For VAE dataset formatting
from torch.utils.data import TensorDataset, DataLoader

# VAE functions
from VAE import Decoder, Encoder, VAE

# Utility file contains all functions required to run notebook
from utils import set_seed, support_pre_proc, plot_elbo, plot_likelihood_breakdown, plot_variable_distributions, reverse_transformers
from metrics import distribution_metrics, privacy_metrics

## Data Loading & Column Definitions

First we load in the SUPPORT dataset from pycox datasets. Then we define the continuous and categorical columns in that dataset

In [2]:
set_seed(0)

# Load in the support data
data_supp = support.read_df()

# Column Definitions
original_continuous_columns = ['duration'] + [f"x{i}" for i in range(7,15)]
original_categorical_columns = ['event'] + [f"x{i}" for i in range(1,7)]

## Data Pre-Processing

Data can be pre-processed in 2 ways. Either we use <b>"standard"</b> option which performs a standard scaler on continuous variables - This has known limitations as:

- Data in tables is usually non-gaussian and SynthVAE implements a gaussian loss, so this will perform worse unless the data is KNOWN to follow a gaussian distribution already.

Or we use the second option of <b>"GMM"</b>. This performs a variational gaussian mixture model to scale the data & transform it to a gaussian distribution. We use a maximum number of clusters of 10 but the variational method will select the best number of clusters for that continuous variable. This also has known limitations:

- 10 Clusters is arbitrary and may not be enough for certain variables.
- We are fitting a model to transform the data and hence we are approximating before model is trained. This will lose fidelity as the distribution will not be transformed perfectly.

SUPPORT is a limited dataset as it has no missingness (which our model currently does NOT handle) and it has no datetime columns or other data types. Be wary drawing any conclusions from this set due to these constraints as well as the dataset size. Testing/training new models with this set could be useful but conclusive results should be tested on other sets.

In [3]:
#%% -------- Data Pre-Processing -------- #

pre_proc_method = "GMM"

x_train, data_supp, reordered_dataframe_columns, continuous_transformers, categorical_transformers, num_categories, num_continuous = support_pre_proc(data_supp=data_supp, pre_proc_method=pre_proc_method)

C:\Users\dxb085\Anaconda3\envs\SynthVAE\lib\site-packages\pandas\core\frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\Users\dxb085\Anaconda3\envs\SynthVAE\lib\site-packages\sklearn\mixture\_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
C:\Users\dxb085\Anaconda3\envs\SynthVAE\lib\site-packages\sklearn\mixture\_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  warnings.warn('Initialization %d did not converge. '
C:\Users\dxb085\Anaconda3\envs\Syn

## Creation & Training of VAE

We can adapt certain parameters of the model e.g. batch size, latent dimension size etc. This model implements early stopping and these values can be adapted.

We can also activate differential privacy by implementing dp-sgd through the opacus library.

In [8]:
#%% -------- Create & Train VAE -------- #

# User defined hyperparams
# General training
batch_size=32
latent_dim=2
hidden_dim=256
n_epochs=50
logging_freq=1 # Number of epochs we should log the results to the user
patience=5 # How many epochs should we allow the model train to see if
# improvement is made
delta=10 # The difference between elbo values that registers an improvement
filepath=None # Where to save the best model


# Privacy params
differential_privacy = True # Do we want to implement differential privacy
sample_rate=0.1 # Sampling rate
C = 1e16 # Clipping threshold - any gradients above this are clipped
noise_scale=None # Noise multiplier - influences how much noise to add
target_eps=1 # Target epsilon for privacy accountant
target_delta=1e-5 # Target delta for privacy accountant

# Prepare data for interaction with torch VAE
Y = torch.Tensor(x_train)
dataset = TensorDataset(Y)

generator = None
sample_rate = batch_size / len(dataset)
data_loader = DataLoader(
    dataset,
    batch_sampler=UniformWithReplacementSampler(
        num_samples=len(dataset), sample_rate=sample_rate, generator=generator
    ),
    pin_memory=True,
    generator=generator,
)

# Create VAE
encoder = Encoder(x_train.shape[1], latent_dim, hidden_dim=hidden_dim)
decoder = Decoder(
    latent_dim, num_continuous, num_categories=num_categories
)

vae = VAE(encoder, decoder)

if(differential_privacy==False):
    log_elbo, log_reconstruction, log_divergence, log_categorical, log_numerical = vae.train(data_loader, n_epochs=n_epochs)
    
elif(differential_privacy==True):
    log_elbo, log_reconstruction, log_divergence, log_categorical, log_numerical = vae.diff_priv_train(
            data_loader,
            n_epochs=n_epochs,
            C=C,
            target_eps=target_eps,
            target_delta=target_delta,
            sample_rate=sample_rate,
            noise_scale=noise_scale
        )
    print(f"(epsilon, delta): {vae.get_privacy_spent(target_delta)}")

c:\Users\dxb085\Documents\NHSX Internship\SynthVAE\Investigations\..\opacus\privacy_engine.py:645: UserWarning: A ``sample_rate`` has been provided.Thus, the provided ``batch_size``and ``sample_size`` will be ignored.
  warnings.warn(
c:\Users\dxb085\Documents\NHSX Internship\SynthVAE\Investigations\..\opacus\privacy_engine.py:229: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  warnings.warn(


Encoder: gpu specified, cuda:0 used
Decoder: gpu specified, cuda:0 used


100%|██████████| 277/277 [00:05<00:00, 50.07it/s]


	Epoch:  0. Elbo:    85724.07. Reconstruction Loss:    85513.07. KL Divergence:      211.00. Categorical Loss:    -7883.67. Numerical Loss:   -77629.40


100%|██████████| 277/277 [00:05<00:00, 53.23it/s]


...

	Epoch: 47. Elbo:   111937.51. Reconstruction Loss:    86290.34. KL Divergence:    25647.17. Categorical Loss:    -8023.97. Numerical Loss:   -78266.37


100%|██████████| 277/277 [00:05<00:00, 51.63it/s]


	Epoch: 48. Elbo:   110564.21. Reconstruction Loss:    83760.34. KL Divergence:    26803.87. Categorical Loss:    -7997.39. Numerical Loss:   -75762.95


100%|██████████| 277/277 [00:05<00:00, 51.06it/s]

	Epoch: 49. Elbo:   115611.67. Reconstruction Loss:    85104.36. KL Divergence:    30507.31. Categorical Loss:    -8005.94. Numerical Loss:   -77098.42
(epsilon, delta): (0.9954940576041075, 18.0)


## Plotting Elbo Functionality

Here we can plot and save the ELBO graph for the training run

In [ ]:
#%% -------- Plot Loss Features ELBO Breakdown -------- #

elbo_fig = plot_elbo(
    n_epochs=26, log_elbo=log_elbo, log_reconstruction=log_reconstruction,
    log_divergence=log_divergence, saving_filepath=""
)

## Plotting Reconstruction Breakdown

Here we can plot the breakdown of reconstruction loss i.e. visualise how the categorical and numerical losses change over training

In [ ]:
#%% -------- Plot Loss Features Reconstruction Breakdown -------- #

likelihood_fig = plot_likelihood_breakdown(
    n_epochs=26, log_categorical=log_categorical, log_numerical=log_numerical,
    saving_filepath="", pre_proc_method=pre_proc_method
)

## Synthetic Data Generation

Here we create synthetic data ready for metric testing as well as visualisation of variable reconstruction. For this we simply generate from our generative model and then reverse transformations using the prior transformers.

In [9]:
#%% -------- Synthetic Data Generation -------- #

synthetic_sample = vae.generate(data_supp.shape[0])

if(torch.cuda.is_available()):
    synthetic_sample = pd.DataFrame(synthetic_sample.cpu().detach(), columns=reordered_dataframe_columns)
else:
    synthetic_sample = pd.DataFrame(synthetic_sample.detach(), columns=reordered_dataframe_columns)

# Reverse the transformations

synthetic_supp = reverse_transformers(synthetic_set=synthetic_sample, data_supp_columns=data_supp.columns, 
                                      cont_transformers=continuous_transformers, cat_transformers=categorical_transformers,
                                      pre_proc_method=pre_proc_method
                                     )

## Synthetic Variable Visualisation

Here we want to visualise the synthetic variables generated and compare them to the original set

In [ ]:
#%% -------- Plot Histograms For All The Variable Distributions -------- #

plot_variable_distributions(
    categorical_columns=original_categorical_columns, continuous_columns=original_continuous_columns,
    data_supp=data_supp, synthetic_supp=synthetic_supp,saving_filepath="",
    pre_proc_method=pre_proc_method
)

## Metric evaluation

We use the SDV evaluation framework. Supply the metrics you wish to find in the user_metrics list from SDV guidance. Can start here: https://sdv.dev/SDV/user_guides/evaluation/single_table_metrics.html

Note that not all of these will work, some are hit and miss. We predominantly rely on continuous and discrete KL divergence measures. You can also input <b>"gower"</b> and this will calculate the gower distance using the gower library.

In [6]:
#%% -------- SDV Metrics -------- #

# Define the metrics you want the model to evaluate

gower=False

metrics = distribution_metrics(
    gower=gower, data_supp=data_supp, synthetic_supp=synthetic_supp,
    categorical_columns=original_categorical_columns, continuous_columns=original_continuous_columns,
    saving_filepath=None, pre_proc_method=pre_proc_method
)

C:\Users\dxb085\Anaconda3\envs\SynthVAE\lib\site-packages\sdmetrics\column_pairs\statistical\kl_divergence.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real_data[pd.isna(real_data)] = 0.0
C:\Users\dxb085\Anaconda3\envs\SynthVAE\lib\site-packages\pandas\core\frame.py:3093: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)
C:\Users\dxb085\Anaconda3\envs\SynthVAE\lib\site-packages\sdmetrics\column_pairs\statistical\kl_divergence.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

# Privacy Metric Evaluation

Using SDV privacy metrics we can get an insight into how privacy is conserved when utilising dp-sgd methods. SDV's privacy metrics are limited in that they can only be used on similar data types. E.g. if we choose age to be the sensitive variably, we can build ML based models to predict a users age using the other columns. However we are forced to only use columns that are also continuous variables.

In [10]:
# Specify our private variable

private_variable = 'x14'

privacy_metric = privacy_metrics(private_variable=private_variable, data_supp=data_supp,
                                synthetic_supp=synthetic_supp, categorical_columns=original_categorical_columns,
                                continuous_columns=original_continuous_columns, saving_filepath=None, pre_proc_method=pre_proc_method)

C:\Users\dxb085\Anaconda3\envs\SynthVAE\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\dxb085\Anaconda3\envs\SynthVAE\lib\site-packages\scipy\stats\_continuous_distns.py:5003: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
C:\Users\dxb085\Anaconda3\envs\SynthVAE\lib\site-packages\scipy\stats\_distn_infrastructure.py:2494: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
C:\Users\dxb085\Anaconda3\envs\SynthVAE\lib\site-packages\copulas\univariate\truncated_gaussian.py:45: RuntimeWarning: divide by zero encountered in double_scalars
  a = (self.min - loc) / scale
C:\Users\dxb085\Anaconda3\envs\SynthVAE\lib\site-packages\copulas\univariate\truncated_gaussian.py:46: RuntimeWarning: divide by zero encountered in double_sca